In [ ]:
# python3 deep_net.py
# Training and Testing a Deep Neural Network on mnist dataset

import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data

#to avoid warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)


mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

n_layers = 3
n_nodes = [784, 500, 500, 500] #input layer has 28*28 input nodes
n_classes = 10
batch_size = 100

x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

def neural_network_model(data):
    hidden_layer = []

    for i in range(n_layers):
        layer = {'weights' : tf.Variable( tf.random_normal( [n_nodes[i],n_nodes[i+1]] )) ,
                 'biases'  : tf.Variable( tf.random_normal( [n_nodes[i+1]] )) }
        hidden_layer.append(layer)

    output_layer = { 'weights' : tf.Variable( tf.random_normal( [n_nodes[n_layers], n_classes] )) ,
                     'biases'  : tf.Variable( tf.random_normal( [n_classes] )) }

    # (input_data*weights) + biases

    layers = []

    layers.append(data)
    for i in range(n_layers):
        l = tf.add(tf.matmul(layers[i],hidden_layer[i]['weights']), hidden_layer[i]['biases'])
        l = tf.nn.relu(l)
        layers.append(l)

    output = tf.matmul(layers[n_layers-1],output_layer['weights']) + output_layer['biases']

    return output

def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    optimiser = tf.train.AdamOptimizer().minimize(cost)

    n_epochs = 10

    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())

        for epoch in range(n_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                _,c = sess.run([optimiser,cost] , feed_dict={x:epoch_x,y:epoch_y})
                epoch_loss += c
            print("Epoch",epoch,"completed out of",n_epochs,"loss",epoch_loss)

        correct = tf.equal(tf.arg_max(prediction,1) , tf.arg_max(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print("Accuracy:",accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))

train_neural_network(x)

tf.logging.set_verbosity(old_v)

